<a href="https://colab.research.google.com/github/NourHenni/Chatbot_BERT/blob/main/DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install transformers[torch]
import nltk
nltk.download('punkt')


# **1.  Importation**



In [ ]:
import json
import pandas as pd
import random
from matplotlib import pyplot as plt
import numpy as np



from sklearn.model_selection import train_test_split  # Importation de train_test_split pour diviser les données en ensembles d'entraînement et de test.
from sklearn.metrics import accuracy_score, precision_recall_fscore_support  # Importation de métriques pour évaluer les modèles.

# Importation des bibliothèques nécessaires pour le prétraitement des données textuelles et l'utilisation des modèles de deep learning.
import nltk  # Bibliothèque pour le traitement du langage naturel
from nltk import word_tokenize  # Fonction de tokenisation des mots
from nltk.stem import PorterStemmer  # Algorithme de racinisation pour le prétraitement des mots

import torch  # Bibliothèque principale pour l'apprentissage automatique
from torch.utils.data import Dataset  # Classe de Dataset pour manipuler les données

# Importation des classes et fonctions spécifiques des Transformers pour l'utilisation des modèles de traitement du langage naturel
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification  # Modèles automatiques et Tokenizers
from transformers import pipeline  # Création de pipelines pour l'inférence avec les modèles Transformers
from transformers import DistilBertTokenizerFast  # Tokenizer rapide pour le modèle DistilBERT
from transformers import BertForSequenceClassification, BertTokenizerFast  # Modèle BERT pour la classification de séquences
from transformers import BertTokenizer, TFBertForSequenceClassification, BertConfig  # Configurations pour BERT
from transformers import Trainer, TrainingArguments  # Entraînement des modèles Transformers avec des arguments spécifiques




# **2. Analyse Exploratoire des données**
### **2.1 Chargement des données**








In [ ]:
def load_json_file(filename):
    with open(filename) as f:
        file = json.load(f)
    return file

filename = '/content/intents.json'

intents = load_json_file(filename)


### **2.2 Extraction des informations du fichier de données Json et stockez-les dans le dataframe**


In [ ]:
def create_df():
    df = pd.DataFrame({
        'Pattern' : [],
        'Tag' : []
    })

    return df

df = create_df()
df

,Pattern,Tag


In [ ]:
def extract_json_info(json_file, df):

    for intent in json_file['intents']:

        for pattern in intent['patterns']:
            # Création d'une liste contenant le pattern et l'étiquette de l'intention
            sentence_tag = [pattern, intent['tag']]
            # Ajout de la liste en tant que nouvelle ligne dans le DataFrame
            df.loc[len(df.index)] = sentence_tag

    # Retour du DataFrame mis à jour avec les informations extraites
    return df

# Appel de la fonction extract_json_info avec les données JSON et un DataFrame vide
df = extract_json_info(intents, df)
df.head()



,Pattern,Tag
0,Hi,Greeting
1,Hi there,Greeting
2,Hola,Greeting
3,Hello,Greeting
4,Hello there,Greeting



### **2.3 Vérification de la forme de l'ensemble de données**

In [ ]:
def print_shape_df(df, ds_name="df"):
    print(f"{ds_name} dataset has {df.shape[0]} rows and {df.shape[1]} columns")

print_shape_df(df, "Chatbot")

Chatbot dataset has 143 rows and 2 columns


### **2.4 Afficher des informations sur l'ensemble de données**

In [ ]:
def print_dfInfo(df, ds_name="df"):
    print(f"The info of {ds_name} dataset\n")
    print(df.info())

print_dfInfo(df, "Chatbot")

The info of Chatbot dataset

<class 'pandas.core.frame.DataFrame'>
Index: 143 entries, 0 to 142
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Pattern  143 non-null    object
 1   Tag      143 non-null    object
dtypes: object(2)
memory usage: 3.4+ KB
None


### **2.5 Afficher le nombre de classes**

In [ ]:
def num_classes(df, target_col, ds_name="df"):
    print(f"The {ds_name} dataset has {len(df[target_col].unique())} classes")

num_classes(df, 'Tag', "Chatbot")

The Chatbot dataset has 22 classes


### **2.6 Vérifiez les valeurs nulles dans l'ensemble de données**

In [ ]:
def check_null(df, ds_name='df'):
    print(f"Null Values in each col in the {ds_name} dataset:\n")
    print(df.isnull().sum())

check_null(df, "Chatbot")

Null Values in each col in the Chatbot dataset:

Pattern    0
Tag        0
dtype: int64


# **3. Prétraitement des données**

In [ ]:
labels = df['Tag'].unique().tolist()
labels = [s.strip() for s in labels]
labels

['Greeting',
 'GreetingResponse',
 'CourtesyGreeting',
 'CourtesyGreetingResponse',
 'CurrentHumanQuery',
 'NameQuery',
 'RealNameQuery',
 'TimeQuery',
 'Thanks',
 'NotTalking2U',
 'UnderstandQuery',
 'Shutup',
 'Swearing',
 'GoodBye',
 'CourtesyGoodBye',
 'WhoAmI',
 'Clever',
 'Gossip',
 'Jokes',
 'PodBayDoor',
 'PodBayDoorResponse',
 'SelfAware']

In [ ]:
#  le nombre de labels dans la liste
num_labels = len(labels)


#  dictionnaire pour mapper les IDs aux libellés
id2label = {id: label for id, label in enumerate(labels)}

#  dictionnaire pour mapper les libellés aux IDs
label2id = {label: id for id, label in enumerate(labels)}


In [ ]:
id2label

{0: 'Greeting',
 1: 'GreetingResponse',
 2: 'CourtesyGreeting',
 3: 'CourtesyGreetingResponse',
 4: 'CurrentHumanQuery',
 5: 'NameQuery',
 6: 'RealNameQuery',
 7: 'TimeQuery',
 8: 'Thanks',
 9: 'NotTalking2U',
 10: 'UnderstandQuery',
 11: 'Shutup',
 12: 'Swearing',
 13: 'GoodBye',
 14: 'CourtesyGoodBye',
 15: 'WhoAmI',
 16: 'Clever',
 17: 'Gossip',
 18: 'Jokes',
 19: 'PodBayDoor',
 20: 'PodBayDoorResponse',
 21: 'SelfAware'}

In [ ]:
label2id

{'Greeting': 0,
 'GreetingResponse': 1,
 'CourtesyGreeting': 2,
 'CourtesyGreetingResponse': 3,
 'CurrentHumanQuery': 4,
 'NameQuery': 5,
 'RealNameQuery': 6,
 'TimeQuery': 7,
 'Thanks': 8,
 'NotTalking2U': 9,
 'UnderstandQuery': 10,
 'Shutup': 11,
 'Swearing': 12,
 'GoodBye': 13,
 'CourtesyGoodBye': 14,
 'WhoAmI': 15,
 'Clever': 16,
 'Gossip': 17,
 'Jokes': 18,
 'PodBayDoor': 19,
 'PodBayDoorResponse': 20,
 'SelfAware': 21}

In [ ]:
#utilisation la méthode map avec une fonction lambda pour mapper chaque libellé de la colonne 'Tag'
# à son identifiant correspondant à l'aide du dictionnaire 'label2id'
df['labels'] = df['Tag'].map(lambda x: label2id[x.strip()])
df.head(15)

,Pattern,Tag,labels
0,Hi,Greeting,0
1,Hi there,Greeting,0
2,Hola,Greeting,0
3,Hello,Greeting,0
4,Hello there,Greeting,0
5,Hya,Greeting,0
6,Hya there,Greeting,0
7,My user is Adam,GreetingResponse,1
8,This is Adam,GreetingResponse,1
9,I am Adam,GreetingResponse,1


# **4. Divisez les données en entrainement et en test**

In [ ]:
#X : Données textuelles à utiliser pour l'entrainement du modèle
X = list(df['Pattern'])
X[:10]

['Hi',
 'Hi there',
 'Hola',
 'Hello',
 'Hello there',
 'Hya',
 'Hya there',
 'My user is Adam',
 'This is Adam',
 'I am Adam']

In [ ]:
# y : Étiquettes ou cibles correspondant aux données dans X
y = list(df['labels'])
y[:10]

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1]

In [ ]:
# Diviser les données en ensembles de formation et de test
# random_state=123 : Utilisé pour assurer la reproductibilité de la division aléatoire
# fonction fournie par la bibliothèque Scikit-Learn divise les données en  (X_train et y_train) et de test (X_test et y_test).
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

#X_train : Les données d'entraînement, c'est-à-dire les textes utilisés pour former le modèle.
#X_test : Les données de test, c'est-à-dire les textes utilisés pour évaluer les performances du modèle après la formation.
#y_train : Les étiquettes correspondant aux données d'entraînement.
#y_test : Les étiquettes correspondant aux données de test.


# **5. Chargement du modèle pré-entraîné BERT et le Tokenizer**

In [ ]:
# Spécifier le nom du modèle BERT à utiliser
model_name = "bert-base-uncased" # C'est le modèle de base de BERT sans distinction entre majuscules et minuscules et sans ajout de caractères spéciaux

# Définir la longueur maximale des séquences à traiter
max_len = 256

# Charger le tokenizer associé au modèle BERT avec la longueur maximale spécifiée
tokenizer = BertTokenizer.from_pretrained(model_name, max_length=max_len)

# Charger le modèle BERT pré-entraîné pour la classification de séquences
model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,  # Nombre de classes à prédire dans la tâche de classification
    id2label=id2label,      # Correspondance d'index à étiquette pour décrire les prédictions
    label2id=label2id       # Correspondance d'étiquette à index pour convertir les étiquettes en indices
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# **6.Transformer les données au format numérique**

In [ ]:
# Utilisation du tokenizer pour encoder les données d'entraînement et de test
# Tronquer les séquences qui dépassent la longueur maximale  et ajouter du padding aux séquences plus courtes
train_encoding = tokenizer(X_train, truncation=True, padding=True)
test_encoding = tokenizer(X_test, truncation=True, padding=True)


In [ ]:
# Utilisation du tokenizer pour traiter toutes les séquences de texte dans la liste X
# Tronquer les séquences de texte qui dépassent la longueur maximale autorisée et ajouter du padding si nécessaire
full_data = tokenizer(X, truncation=True, padding=True)


# **7.Construire un chargeur de données**

In [ ]:
#classe DataLoader qui hérite de Dataset de PyTorch. Elle est utilisée pour manipuler les données encodées et leurs étiquettes associées
#pour l'entrainement du modéle
class DataLoader(Dataset):
    # Constructeur de la classe DataLoader
    def __init__(self, encodings, labels):
        # Initialise les attributs encodings et labels de l'instance avec les données fournies
        self.encodings = encodings
        self.labels = labels

    # Méthode spéciale pour accéder à un élément de l'ensemble de données par indice
    def __getitem__(self, idx):
        # Crée un dictionnaire contenant les données encodées pour l'élément à l'indice idx
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Ajoute les étiquettes associées à cet élément dans le dictionnaire
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    # Méthode  pour obtenir la longueur totale de l'ensemble de données
    def __len__(self):
        # Renvoie le nombre d'éléments dans l'ensemble de données, qui est égal au nombre de labels
        return len(self.labels)


In [ ]:
# création de 2 instances de la classe DataLoader
train_dataloader = DataLoader(train_encoding, y_train)
test_dataloader = DataLoader(test_encoding, y_test)

In [ ]:
fullDataLoader = DataLoader(full_data, y_test)

# **8.Définir les meusures d'évaluation**

In [ ]:
def compute_metrics(pred):

    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)

    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision, # est une mesure de la qualité d'un modèle de classification
        'Recall': recall #est une mesure de la capacité d'un modèle de classification à identifier correctement tous les exemples positifs
    }

# **9.Définir les hyperparamètres & Entraînement du model**







In [ ]:
from transformers import Trainer, TrainingArguments
# trainer est une classe qui permet de gérer le processus d'entraînement d'un modèle
# TrainingArguments est une classe qui contient les arguments et les hyperparamètres pour l'entraînement d'un modèle

# Création des arguments d'entraînement
training_args = TrainingArguments(
    output_dir='./output',  # Répertoire de sortie pour sauvegarder les modèles et les logs d'entraînement
    do_train=True,  # Indique si l'entraînement doit être effectué
    do_eval=True,  # Indique si l'évaluation doit être effectuée après chaque époque d'entraînement
    num_train_epochs=60,  # Nombre total d'époques pour l'entraînement
    per_device_train_batch_size=32,  # Taille du lot par périphérique pour l'entraînement
    per_device_eval_batch_size=16,  # Taille du lot par périphérique pour l'évaluation
    warmup_steps=100,  # Nombre d'étapes d'échauffement pour l'ajustement de l'apprentissage
    weight_decay=0.05,  # Paramètre de régularisation pour la décroissance du poids
    logging_strategy='steps',  # Stratégie de journalisation des métriques d'entraînement
    logging_dir='./multi-class-logs',  # Répertoire pour enregistrer les fichiers de journalisation
    logging_steps=100,  # Fréquence des étapes pour enregistrer les métriques de journalisation
    evaluation_strategy="steps",  # Stratégie d'évaluation pour l'évaluation périodique
    eval_steps=100,  # Fréquence des étapes pour l'évaluation du modèle
    save_strategy="steps",  # Stratégie de sauvegarde pour sauvegarder le modèle à chaque étape
    load_best_model_at_end=True  # Chargement du meilleur modèle à la fin de l'entraînement
)


# Création une instance de Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataloader,  # DataLoader pour l'entraînement
    eval_dataset=test_dataloader,  #  DataLoader pour l'évaluation
    compute_metrics=compute_metrics  # Fonction pour calculer les métriques
)

# Lancer l'entraînement
trainer.train()


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,2.217500,1.271808,0.805556,0.781955,0.802632,0.833333
200,0.160900,0.767699,0.833333,0.848622,0.846491,0.859649


TrainOutput(global_step=240, training_loss=0.9975152711073557, metrics={'train_runtime': 903.9651, 'train_samples_per_second': 7.102, 'train_steps_per_second': 0.265, 'total_flos': 36297342457200.0, 'train_loss': 0.9975152711073557, 'epoch': 60.0})

# **10. Evaluation du model**




In [ ]:
q=[trainer.evaluate(eval_dataset=df) for df in [train_dataloader, test_dataloader]]

pd.DataFrame(q, index=["train","test"]).iloc[:,:5]

,eval_loss,eval_Accuracy,eval_F1,eval_Precision,eval_Recall
train,0.027102,1.000000,1.000000,1.000000,1.000000
test,0.786480,0.805556,0.813534,0.820175,0.833333


In [ ]:
def predict(text):
    # Tokeniser le texte et le transformer en tensors PyTorch
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cpu")
    #  prédiction avec le modèle
    outputs = model(**inputs)
    # Appliquation de la fonction softmax pour obtenir les probabilités des classes
    probs = outputs[0].softmax(1)
    # Déterminer l'indice de la classe prédite
    pred_label_idx = probs.argmax()
    # Obtenir l'étiquette prédite à partir de l'indice de classe
    pred_label = model.config.id2label[pred_label_idx.item()]
    # Retourne les probabilités, l'indice de l'étiquette prédite et l'étiquette prédite
    return probs, pred_label_idx, pred_label


# **11. Enregistrement du model**

In [ ]:
model_path = "chatbot"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)
# contient la configuration du tokenizer #chatbot/tokenizer_config.json
#Ce fichier cartographie les tokens spéciaux aux identifiants numériques dans le tokenizer. #chatbot/special_tokens_map.jso
# Ce fichier contient le vocabulaire du tokenizer, avec chaque token sur une ligne distincte et son identifiant numérique correspondant #chatbot/vocab.txt
# Ce fichier contient des tokens spéciaux ajoutés au vocabulaire existant du tokenizer, avec leurs identifiants numériques #chatbot/added_tokens.json

('chatbot/tokenizer_config.json',
 'chatbot/special_tokens_map.json',
 'chatbot/vocab.txt',
 'chatbot/added_tokens.json')

# **12. Test du Chatbot**

In [ ]:
# Définition du chemin du modèle et chargement du modèle BERT pour l'analyse de sentiment
model_path = "chatbot"
model = BertForSequenceClassification.from_pretrained(model_path)

# Chargement du tokenizer correspondant au modèle
tokenizer = BertTokenizerFast.from_pretrained(model_path)

# Création d'un pipeline de classification de sentiment utilisant le modèle et le tokenizer chargés
chatbot = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Définition de la fonction de chat
def chat(chatbot):
    # Message d'introduction pour l'utilisateur
    print("Chatbot: Hi! I am your virtual assistant. Feel free to ask, and I'll do my best to provide you with answers and assistance.")
    print("Type 'quit' to exit the chat\n\n")

    # Demande de l'entrée de l'utilisateur
    text = input("User: ").strip().lower()


    while text != 'quit':
        # Obtention de la prédiction de sentiment pour l'entrée de l'utilisateur
        prediction = chatbot(text)[0]
        # Conversion de l'étiquette prédite en identifiant correspondant
        label = label2id[chatbot(text)[0]['label']]
        # Récupération du score de confiance de la prédiction
        score = prediction['score']

        # Vérification de la confiance de la prédiction
        if score < 0.8:
            # Si la confiance est faible, afficher un message d'incertitude
            print("Chatbot: Sorry, I can't answer that\n\n")
        else:
            # Si la confiance est suffisante,
            response = random.choice(intents['intents'][label]['responses'])
            # Affichage de la réponse
            print(f"Chatbot: {response}\n\n")


        text = input("User: ").strip().lower()

# Appel de la fonction de chat avec le chatbot initialisé pour démarrer l'interaction
chat(chatbot)


Chatbot: Hi! I am your virtual assistant. Feel free to ask, and I'll do my best to provide you with answers and assistance.
Type 'quit' to exit the chat


User: hello
Chatbot: Hi human, please tell me your GeniSys user


User: how are you
Chatbot: Hi, I am good thank you, how are you? Please tell me your GeniSys user


User: fine thanks
Chatbot: My pleasure


User: tell me a joke
Chatbot: Man: (to friend) I'm taking my wife on an African Safari. Friend: Wow! What would you do if a vicious lion attacked your wife? Man: Nothing. Friend: Nothing? You wouldn't do anything? Man: Too right. I'd let the stupid lion fend for himself!


User: i ' am nour
Chatbot: OK! Hola <HUMAN>, how can I help you?


User: your name
Chatbot: Sorry, I can't answer that


User: time
Chatbot: One second


User: bye
Chatbot: Have a nice day


User: quit
